<a href="https://colab.research.google.com/github/thrcle/TensorflowMLBasic_ArrivalTimePrediction_SeoulToBusan/blob/main/3_2_6_multi_regression(multi_output).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 집계시간과 요일을 입력받아 135km지점과 407km의 예측시간 찾기

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/fc/data/data_2020.csv")

In [ ]:
df

,집계일자,집계시,출발영업소코드,도착영업소코드,통행시간,요일
0,20200101,4,101,105,637,2
1,20200101,4,101,105,773,2
2,20200101,4,101,105,762,2
3,20200101,4,101,105,746,2
4,20200101,4,101,105,875,2
...,...,...,...,...,...,...
152865,20201231,3,101,130,8806,3
152866,20201231,5,101,130,8733,3
152867,20201231,6,101,110,3173,3
152868,20201231,13,101,110,3866,3


In [ ]:
df['거리']=df['도착영업소코드'].map({105: 20, 110: 77, 115: 135, 120: 185, 125: 240, 130: 215, 135: 339, 140: 407})
df.pop('도착영업소코드')
df

,집계일자,집계시,출발영업소코드,통행시간,요일,거리
0,20200101,4,101,637,2,20
1,20200101,4,101,773,2,20
2,20200101,4,101,762,2,20
3,20200101,4,101,746,2,20
4,20200101,4,101,875,2,20
...,...,...,...,...,...,...
152865,20201231,3,101,8806,3,215
152866,20201231,5,101,8733,3,215
152867,20201231,6,101,3173,3,77
152868,20201231,13,101,3866,3,77


In [ ]:
df2=df[df['거리'].isin([135,407])]  #isin : 복수개의 대상을 뽑을 때 유용
df2

,집계일자,집계시,출발영업소코드,통행시간,요일,거리
174,20200101,10,101,5499,2,135
175,20200101,10,101,5329,2,135
176,20200101,20,101,5673,2,135
177,20200101,20,101,4662,2,135
178,20200101,4,101,5946,2,135
...,...,...,...,...,...,...
152857,20201231,17,101,15328,3,407
152860,20201231,2,101,4539,3,135
152861,20201231,20,101,17001,3,407
152862,20201231,6,101,4652,3,135


In [ ]:
# 집계시간과 요일을 기준으로 그룹화 -> 통행시간의 평균값

df3=df2.groupby(['집계시','요일','거리'])['통행시간'].mean()
df3

집계시  요일  거리 
0    0   135     5072.637037
         407    15549.675676
     1   135     5051.017391
         407    15983.102564
     2   135     5450.725000
                    ...     
23   4   407    17035.450980
     5   135     5431.022556
         407    16033.622222
     6   135     5254.222973
         407    15297.022727
Name: 통행시간, Length: 336, dtype: float64

In [ ]:
df4=df3.unstack().dropna()  #level=-1
df4

거리              135           407
집계시 요일                           
0   0   5072.637037  15549.675676
    1   5051.017391  15983.102564
    2   5450.725000  15129.750000
    3   5263.705882  14533.189189
    4   5219.400000  15908.275000
...             ...           ...
23  2   5404.398601  16006.125000
    3   5627.057554  16118.288889
    4   6017.818182  17035.450980
    5   5431.022556  16033.622222
    6   5254.222973  15297.022727

[168 rows x 2 columns]

In [ ]:
df5=df4.reset_index()
df5

거리,집계시,요일,135,407
0,0,0,5072.637037,15549.675676
1,0,1,5051.017391,15983.102564
2,0,2,5450.725000,15129.750000
3,0,3,5263.705882,14533.189189
4,0,4,5219.400000,15908.275000
...,...,...,...,...
163,23,2,5404.398601,16006.125000
164,23,3,5627.057554,16118.288889
165,23,4,6017.818182,17035.450980
166,23,5,5431.022556,16033.622222


In [ ]:
df6=df5.values.tolist()

In [ ]:
train_x=[r[:2] for r in df6]
train_y=[r[2:] for r in df6]

In [ ]:
# tensorflow

learning_rate = 1e-4
learning_epochs = 5000

# Stochastic gradient descent (SGD) Optimizer
sgd =  tf.keras.optimizers.SGD(learning_rate=learning_rate)
# Mean Square Error (MSE) loss function
mse = tf.keras.losses.mean_squared_error

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(2, input_shape=(2,)))

model.compile(loss=mse, optimizer=sgd)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 2)                 6         
                                                                 
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(train_x, train_y, epochs=learning_epochs)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
6/6 [==============================] - 0s 3ms/step - loss: 12920027.0000
Epoch 2502/5000
6/6 [==============================] - 0s 3ms/step - loss: 12919953.0000
Epoch 2503/5000
6/6 [==============================] - 0s 3ms/step - loss: 12918278.0000
Epoch 2504/5000
6/6 [==============================] - 0s 3ms/step - loss: 12917723.0000
Epoch 2505/5000
6/6 [==============================] - 0s 3ms/step - loss: 12910202.0000
Epoch 2506/5000
6/6 [==============================] - 0s 3ms/step - loss: 12908050.0000
Epoch 2507/5000
6/6 [==============================] - 0s 3ms/step - loss: 12904820.0000
Epoch 2508/5000
6/6 [==============================] - 0s 3ms/step - loss: 12903226.0000
Epoch 2509/5000
6/6 [==============================] - 0s 3ms/step - loss: 12900503.0000
Epoch 2510/5000
6/6 [==============================] - 0s 3ms/step - loss: 12901143.0000
Epoch 2511/5000
6/6 [==============================] - 0s 4ms/step - loss: 12894071.0000
E

In [ ]:
# 집계시와 요일 설정에 따른 실제 거리가 135, 407인 지점의 평균 통행시간 구하기

time=14 #@param:{type:'slider', min:0, max:23, step:1}
day=3 #@param:{type:'slider', min:0, max:6, step:1}



In [ ]:


df5[(df5['집계시']==time) & (df5['요일']==day)]

거리,집계시,요일,135,407
101,14,3,6077.381503,15936.189655


In [ ]:
# 예측값 구하기

model.predict([[time, day]])

array([[ 5524.1797, 15285.264 ]], dtype=float32)

In [ ]:
# 이 case의 경우 train set과 test set을 나누지 않고 진행했음